In [15]:
#from transformers import AutoProcessor,AutoModelForCTC,AutoModelForSeq2SeqLM  -> 정확히 매칭 되야함
from transformers import AutoProcessor, AutoModelForSpeechSeq2Seq # 이 부분을 변경!


In [20]:
# Wav2Vec2 예시 (CTC):
model_name="facebook/wav2vec2-base-960h"
processor_wav=AutoProcessor.from_pretrained(model_name)
model_CTC=AutoModelForCTC.from_pretrained(model_name)


# Whisper 예시 (Seq2Seq):
model_wisper="openai/whisper-small"
processor_wisper= AutoProcessor.from_pretrained(model_wisper)
#model_wis= AutoModelForSeq2SeqLM.from_pretrained(model_wisper)
model_wis= AutoModelForSpeechSeq2Seq.from_pretrained(model_wisper) # <-- 이렇게 사용해야 합니다.


print(f"Wav2Vec2 모델 로드 완료: {model_CTC.__class__.__name__}")
print(f"Whisper 모델 로드 완료: {model_wis.__class__.__name__}")

Some weights of Wav2Vec2ForCTC were not initialized from the model checkpoint at facebook/wav2vec2-base-960h and are newly initialized: ['wav2vec2.masked_spec_embed']
You should probably TRAIN this model on a down-stream task to be able to use it for predictions and inference.


Wav2Vec2 모델 로드 완료: Wav2Vec2ForCTC
Whisper 모델 로드 완료: WhisperForConditionalGeneration


In [25]:
import torch
import torch.nn as nn
import torch.optim as optim
new_vocab=30
model_CTC.lm_head=nn.Linear(model_CTC.config.hidden_size, new_vocab)
print(f"Wav2Vec2의 새로운 lm_head 출력 차원: {model_CTC.lm_head.out_features}")

Wav2Vec2의 새로운 lm_head 출력 차원: 30


In [27]:
lr=1e-5
optimy=optim.AdamW(model_CTC.parameters(),lr=lr)
print(f"옵티마이저 설정 완료. 학습률: {lr}")


옵티마이저 설정 완료. 학습률: 1e-05


In [267]:
from transformers import AutoProcessor, AutoModelForCTC, Wav2Vec2CTCTokenizer
import torch.nn as nn
import torch.optim as optim
from torch.utils.data import DataLoader, Dataset
import torchaudio
from tqdm.auto import tqdm

# --- 0. 환경 설정 및 장치(Device) 설정 ---
device = torch.device('cuda' if torch.cuda.is_available() else 'cpu')
print(f"Using device: {device}")

# 사용할 모델 이름
model_name = "facebook/wav2vec2-base-960h"

Using device: cuda


In [269]:
vocab_list = [
    " ", "|", # 공백과 파이프는 CTC에 자주 사용됨
    # 한글 자음
    "ㄱ", "ㄴ", "ㄷ", "ㄹ", "ㅁ", "ㅂ", "ㅅ", "ㅇ", "ㅈ", "ㅊ", "ㅋ", "ㅌ", "ㅍ", "ㅎ",
    "ㄲ", "ㄸ", "ㅃ", "ㅆ", "ㅉ",
    # 한글 모음
    "ㅏ", "ㅑ", "ㅓ", "ㅕ", "ㅗ", "ㅛ", "ㅜ", "ㅠ", "ㅡ", "ㅣ", "ㅐ", "ㅔ", "ㅚ", "ㅟ", "ㅢ",
    "ㅘ", "ㅝ", "ㅙ", "ㅞ",
    # 숫자 및 기타 기호 (필요시 추가)
    "0", "1", "2", "3", "4", "5", "6", "7", "8", "9",
    ",", ".", "?", "!", "-",
    # 더미 데이터셋에 사용된 문자도 명시적으로 추가
    # "가", "나", "다", "라", "마", "바", "사", "아", "자", "차", "카", "타", "파", "하"
    # 실제 ASR에서는 위 자모 목록이면 충분합니다.
]

# <unk> (알 수 없는 토큰), <pad> (패딩 토큰) 추가
# Wav2Vec2CTCTokenizer는 기본적으로 pad_token, unk_token, word_delimiter_token을 가집니다.
# add_tokens는 이미 어휘에 없는 토큰만 추가합니다.
vocab_list = sorted(list(set(vocab_list))) # 중복 제거 및 정렬

# Vocab 파일 저장 (Tokenization 과정에서 필요)
vocab_dict = {token: i for i, token in enumerate(vocab_list)}

class DummyAudioDataset(Dataset):
    def __init__(self, num_samples=100, audio_length_sec=2, sample_rate=16000, processor=None):
        self.num_samples = num_samples
        self.audio_length_sec = audio_length_sec
        self.sample_rate = sample_rate
        self.processor = processor

        base_chars = "가나다라마바사아자차카타파하" # 더미 텍스트 생성에 사용될 문자
        self.texts = []
        for _ in range(num_samples):
            text_len = torch.randint(5, 15, (1,)).item()
            random_text = ""
            for _ in range(text_len):
                char_idx = torch.randint(0, len(base_chars), (1,)).item()
                random_text += base_chars[char_idx]
            self.texts.append(random_text)

        self.audio_data = [torch.randn(audio_length_sec * sample_rate) for _ in range(num_samples)]

    def __len__(self):
        return self.num_samples

    def __getitem__(self, idx):
        audio_waveform = self.audio_data[idx]
        text_label = self.texts[idx]

        input_values = self.processor.feature_extractor(audio_waveform.unsqueeze(0), sampling_rate=self.sample_rate).input_values[0]

        with self.processor.as_target_processor():
            input_ids = self.processor.tokenizer(text_label).input_ids

        return {"input_values": input_values, "labels": input_ids}

In [270]:
def data_collator(features):
    input_features = [{"input_values": feature["input_values"]} for feature in features]
    batch = processor.feature_extractor.pad(
        input_features,
        padding=True,
        return_tensors="pt"
    )

    label_features = [{"input_ids": feature["labels"]} for feature in features]
    labels_batch = processor.tokenizer.pad(
        label_features,
        padding=True,
        return_tensors="pt"
    )

    labels = labels_batch["input_ids"].masked_fill(labels_batch.attention_mask.ne(1), -100)

    batch["labels"] = labels
    return batch

In [274]:
import json
with open("vocab.json", "w", encoding="utf-8") as f:
    json.dump(vocab_dict, f, ensure_ascii=False)


tokenizer = Wav2Vec2CTCTokenizer(
    "vocab.json", # 위에서 생성한 vocab.json 파일 경로
    unk_token="<unk>",
    pad_token="<pad>",
    word_delimiter_token=" " # 공백을 단어 구분자로
)


In [280]:
from transformers import Wav2Vec2FeatureExtractor
from transformers import Wav2Vec2Processor # Import the specific processor class
feature_extractor = Wav2Vec2FeatureExtractor(
    feature_size=1,
    sampling_rate=16000,
    padding_value=0.0,
    do_normalize=True,
    return_attention_mask=True
)



processor = Wav2Vec2Processor(feature_extractor=feature_extractor, tokenizer=tokenizer)

# 최종 어휘 크기 확인
final_vocab_size = processor.tokenizer.vocab_size
print(f"Processor 직접 구성 및 토큰 설정 완료. 최종 Vocab Size: {final_vocab_size}")

Processor 직접 구성 및 토큰 설정 완료. 최종 Vocab Size: 55


In [290]:
# --- 2. 사전 학습 모델 로드 및 lm_head 수정 (Wav2Vec2 모델용) ---
model = AutoModelForCTC.from_pretrained(
    model_name,
    ctc_loss_reduction="mean",
    pad_token_id=processor.tokenizer.pad_token_id,
    vocab_size=final_vocab_size, # 모델의 config에 새로운 vocab_size를 알려줌
    ignore_mismatched_sizes=True # <--- 이 부분이 핵심! 크기가 다른 레이어의 가중치 로드를 무시합니다.
)

# lm_head를 최종 vocab_size에 맞게 재정의
model.lm_head = nn.Linear(model.config.hidden_size, final_vocab_size)

# 모델을 GPU/CPU로 이동
model.to(device)
print(f"모델 '{model_name}' 로드 및 lm_head 수정 완료. lm_head.out_features: {model.lm_head.out_features}")

Some weights of Wav2Vec2ForCTC were not initialized from the model checkpoint at facebook/wav2vec2-base-960h and are newly initialized: ['wav2vec2.masked_spec_embed']
You should probably TRAIN this model on a down-stream task to be able to use it for predictions and inference.
Some weights of Wav2Vec2ForCTC were not initialized from the model checkpoint at facebook/wav2vec2-base-960h and are newly initialized because the shapes did not match:
- lm_head.bias: found shape torch.Size([32]) in the checkpoint and torch.Size([55]) in the model instantiated
- lm_head.weight: found shape torch.Size([32, 768]) in the checkpoint and torch.Size([55, 768]) in the model instantiated
You should probably TRAIN this model on a down-stream task to be able to use it for predictions and inference.


모델 'facebook/wav2vec2-base-960h' 로드 및 lm_head 수정 완료. lm_head.out_features: 55


In [292]:
# --- 4. 데이터셋 인스턴스 및 DataLoader 생성 ---
train_set = DummyAudioDataset(num_samples=100, processor=processor)

train_loader = DataLoader(
    train_set,
    batch_size=4,
    shuffle=True,
    collate_fn=data_collator
)
print(f"학습 데이터셋 크기: {len(train_set)} 샘플, 배치 개수: {len(train_loader)}")

# 학습 환경 설정 (옵티마이저)
lr = 1e-4
optimy = optim.AdamW(model.parameters(), lr=lr)
print(f"옵티마이저 설정 완료 (AdamW, LR={lr})")

학습 데이터셋 크기: 100 샘플, 배치 개수: 25
옵티마이저 설정 완료 (AdamW, LR=0.0001)


In [294]:
# --- 6. 학습 루프 실행 ---
num_epochs = 3

model.train()

print("\n--- 학습 시작 ---")
for epoch in range(num_epochs):
    total_loss = 0
    for batch_idx, batch in enumerate(tqdm(train_loader, desc=f"Epoch {epoch+1}")):
        input_val = batch["input_values"].to(device)
        label = batch["labels"].to(device)

        optimy.zero_grad()

        out = model(input_values=input_val, labels=label)
        loss = out.loss

        loss.backward()
        optimy.step()

        total_loss += loss.item()

    avg_loss = total_loss / len(train_loader)
    print(f"Epoch {epoch+1}/{num_epochs}, Average Loss: {avg_loss:.4f}")

print("\n--- 학습 완료 ---")


--- 학습 시작 ---


Epoch 1:   0%|          | 0/25 [00:00<?, ?it/s]


ValueError: type of None unknown: <class 'NoneType'>. Should be one of a python, numpy, pytorch or tensorflow object.

In [3]:
!pip install hangul-jamo

In [7]:
# --- (선택 사항) 학습된 모델 저장 ---
#model_save_path = "./my_finetuned_wav2vec2"
#model.save_pretrained(model_save_path)
#processor.save_pretrained(model_save_path)
#print(f"모델 저장 완료: {model_save_path}")

In [35]:
import torch
import torch.nn as nn
import torch.optim as optim
from torch.utils.data import DataLoader, Dataset
import torchaudio
from transformers import AutoProcessor, AutoModelForCTC, Wav2Vec2CTCTokenizer, Wav2Vec2FeatureExtractor, Wav2Vec2Processor
from tqdm.auto import tqdm
import json
import unicodedata

# --- 0. Environment and Device Setup ---
device = torch.device('cuda' if torch.cuda.is_available() else 'cpu')
print(f"Using device: {device}")

# Model name to use
model_name = "facebook/wav2vec2-base-960h"

# --- 1. Custom Tokenizer Creation for Wav2Vec2 Model (MODIFIED VOCAB) ---
# Ensure special tokens are at the end to guarantee their IDs
# are within the valid range [0, vocab_size - 1]
vocab_elements = [
    # Korean Consonants (Jamo)
    "ㄱ", "ㄴ", "ㄷ", "ㄹ", "ㅁ", "ㅂ", "ㅅ", "ㅇ", "ㅈ", "ㅊ", "ㅋ", "ㅌ", "ㅍ", "ㅎ",
    "ㄲ", "ㄸ", "ㅃ", "ㅆ", "ㅉ",
    # Korean Vowels (Jamo)
    "ㅏ", "ㅑ", "ㅓ", "ㅕ", "ㅗ", "ㅛ", "ㅜ", "ㅠ", "ㅡ", "ㅣ", "ㅐ", "ㅔ", "ㅚ", "ㅟ", "ㅢ",
    "ㅘ", "ㅝ", "ㅙ", "ㅞ",
    # Numbers (optional, add if your data includes them)
    "0", "1", "2", "3", "4", "5", "6", "7", "8", "9",
    # Punctuation (optional)
    ",", ".", "?", "!", "-",
    # Important: Put space, word delimiter, and special tokens at the end
    # to ensure their IDs are correctly handled relative to vocab_size
    " ", "|", "<unk>", "<pad>" # Ensure <pad> is present
]

# Create a unique, sorted list, then make sure special tokens are at the very end
unique_vocab = sorted(list(set([v for v in vocab_elements if v not in [" ", "|", "<unk>", "<pad>"]])))
final_vocab_list = unique_vocab + [" ", "|", "<unk>", "<pad>"] # Explicitly add special tokens at the end

# Create vocab_dict with IDs based on their index in the final list
vocab_dict = {token: i for i, token in enumerate(final_vocab_list)}

# Save vocab.json (required by Wav2Vec2CTCTokenizer)
with open("vocab.json", "w", encoding="utf-8") as f:
    json.dump(vocab_dict, f, ensure_ascii=False)

# Load Wav2Vec2CTCTokenizer
# Crucially, ensure the pad_token_id is correctly mapped from the vocab_dict
tokenizer = Wav2Vec2CTCTokenizer(
    "vocab.json",
    unk_token="<unk>",
    pad_token="<pad>",
    word_delimiter_token=" "
)

# Wav2Vec2FeatureExtractor creation
feature_extractor = Wav2Vec2FeatureExtractor(
    feature_size=1, # 1 for mono audio
    sampling_rate=16000,
    padding_value=0.0,
    do_normalize=True,
    return_attention_mask=False # The model typically generates attention mask internally
)

# Combine Feature Extractor and Tokenizer into a Processor
processor = Wav2Vec2Processor(feature_extractor=feature_extractor, tokenizer=tokenizer)

# Verify final vocabulary size
final_vocab_size = processor.tokenizer.vocab_size
print(f"Processor configured and tokens set. Final Vocab Size: {final_vocab_size}")
print(f"Processor's pad_token_id: {processor.tokenizer.pad_token_id}")
print(f"Is pad_token_id within vocab_size range? {processor.tokenizer.pad_token_id < final_vocab_size}")


# --- 2. Load Pre-trained Model and Modify lm_head (for Wav2Vec2 model) ---
model = AutoModelForCTC.from_pretrained(
    model_name,
    ctc_loss_reduction="mean",
    pad_token_id=processor.tokenizer.pad_token_id, # This is correct, ensuring model uses our pad_token_id
    vocab_size=final_vocab_size, # Pass the final_vocab_size to the model config
    ignore_mismatched_sizes=True # Ignore size mismatches (e.g., lm_head) when loading
)

# Redefine lm_head to match the final_vocab_size.
# This ensures the output layer matches your custom vocabulary.
model.lm_head = nn.Linear(model.config.hidden_size, final_vocab_size)

model.to(device)
print(f"Model '{model_name}' loaded and lm_head modified. lm_head.out_features: {model.lm_head.out_features}")

# --- 3. Define DummyAudioDataset Class ---
class DummyAudioDataset(Dataset):
    def __init__(self, num_samples=100, audio_length_sec=2, sample_rate=16000, processor=None):
        self.num_samples = num_samples
        self.audio_length_sec = audio_length_sec
        self.sample_rate = sample_rate
        self.processor = processor

        jamo_chars = [char for char in final_vocab_list if char not in ["<unk>", "<pad>"]] # Exclude special tokens that aren't for generation

        self.texts = []
        for _ in range(num_samples):
            text_len = torch.randint(5, 15, (1,)).item()
            random_text_jamo = ""
            for _ in range(text_len):
                char_idx = torch.randint(0, len(jamo_chars), (1,)).item()
                random_text_jamo += jamo_chars[char_idx]
            
            # Fallback for empty strings (should be rare with jamo_chars including space)
            if not random_text_jamo.strip():
                # Ensure fallback is also tokenizable by your defined vocab
                fallback_jamo = "안녕" # Simpler fallback using just "안녕"
                random_text_jamo = "".join(unicodedata.normalize("NFD", char) for char in fallback_jamo if unicodedata.category(char) != 'Mn')
            
            self.texts.append(random_text_jamo)

        self.audio_data = [torch.randn(audio_length_sec * sample_rate) for _ in range(num_samples)]

    def __len__(self):
        return self.num_samples

    def __getitem__(self, idx):
        audio_waveform = self.audio_data[idx]
        text_label = self.texts[idx]
        return {"audio": audio_waveform, "text": text_label}

# --- 4. Data Collator Function ---
class DataCollatorCTCWithPadding:
    def __init__(self, processor):
        self.processor = processor

    def __call__(self, features):
        audio_samples = [feature["audio"] for feature in features]
        text_labels = [feature["text"] for feature in features]

        padded_audio_batch = self.processor.feature_extractor.pad(
            {"input_values": audio_samples},
            return_tensors="pt",
            padding=True
        )
        batch_input_values = padded_audio_batch["input_values"]

        with self.processor.as_target_processor():
            tokenized_labels = []
            for label in text_labels:
                encoded_label = self.processor.tokenizer(label).input_ids
                if encoded_label:
                    tokenized_labels.append(encoded_label)
                else:
                    print(f"Warning: Empty tokenized label for text: '{label}'. Using default.")
                    default_text = "안녕" # Fallback text
                    decomposed_default_text = "".join(unicodedata.normalize("NFD", char) for char in default_text if unicodedata.category(char) != 'Mn')
                    tokenized_labels.append(self.processor.tokenizer(decomposed_default_text).input_ids)

            labels_batch = self.processor.tokenizer.pad(
                {"input_ids": tokenized_labels},
                padding=True,
                return_tensors="pt"
            )

        labels = labels_batch["input_ids"].masked_fill(labels_batch.attention_mask.ne(1), -100)

        batch = {
            "input_values": batch_input_values,
            "labels": labels
        }
        return batch

# --- 5. Create Dataset Instance and DataLoader ---
train_set = DummyAudioDataset(num_samples=100, processor=processor)

data_collator_instance = DataCollatorCTCWithPadding(processor=processor)

train_loader = DataLoader(
    train_set,
    batch_size=4,
    shuffle=True,
    collate_fn=data_collator_instance
)
print(f"Training dataset size: {len(train_set)} samples, Number of batches: {len(train_loader)}")

# --- 6. Training Environment Setup (Optimizer) ---
lr = 1e-4
optimizer = optim.AdamW(model.parameters(), lr=lr)
print(f"Optimizer configured (AdamW, LR={lr})")

# --- 7. Execute Training Loop ---
num_epochs = 3

model.train()

print("\n--- Training Started ---")
for epoch in range(num_epochs):
    total_loss = 0
    for batch_idx, batch in enumerate(tqdm(train_loader, desc=f"Epoch {epoch+1}")):
        input_val = batch["input_values"].to(device)
        label = batch["labels"].to(device)
        
        optimizer.zero_grad()
        outputs = model(input_values=input_val, labels=label)
        loss = outputs.loss

        loss.backward()
        optimizer.step()

        total_loss += loss.item()

    avg_loss = total_loss / len(train_loader)
    print(f"Epoch {epoch+1}/{num_epochs}, Average Loss: {avg_loss:.4f}")

print("\n--- Training Completed ---")

Using device: cuda
Processor configured and tokens set. Final Vocab Size: 57
Processor's pad_token_id: 56
Is pad_token_id within vocab_size range? True


Some weights of Wav2Vec2ForCTC were not initialized from the model checkpoint at facebook/wav2vec2-base-960h and are newly initialized: ['wav2vec2.masked_spec_embed']
You should probably TRAIN this model on a down-stream task to be able to use it for predictions and inference.
Some weights of Wav2Vec2ForCTC were not initialized from the model checkpoint at facebook/wav2vec2-base-960h and are newly initialized because the shapes did not match:
- lm_head.bias: found shape torch.Size([32]) in the checkpoint and torch.Size([57]) in the model instantiated
- lm_head.weight: found shape torch.Size([32, 768]) in the checkpoint and torch.Size([57, 768]) in the model instantiated
You should probably TRAIN this model on a down-stream task to be able to use it for predictions and inference.


Model 'facebook/wav2vec2-base-960h' loaded and lm_head modified. lm_head.out_features: 57
Training dataset size: 100 samples, Number of batches: 25
Optimizer configured (AdamW, LR=0.0001)

--- Training Started ---


Epoch 1: 100%|██████████| 25/25 [00:01<00:00, 24.37it/s]


Epoch 1/3, Average Loss: 26.3305


Epoch 2: 100%|██████████| 25/25 [00:00<00:00, 30.00it/s]


Epoch 2/3, Average Loss: 8.9754


Epoch 3: 100%|██████████| 25/25 [00:00<00:00, 32.27it/s]

Epoch 3/3, Average Loss: 5.0271

--- Training Completed ---


--

### 1. `ValueError: Expected 2D (unbatched) or 3D (batched) input to conv1d, but got input of size: [4, 1, 1, 1, 32000]`

**문제:** 이 오류는 Wav2Vec2 모델의 내부 `Conv1d` 레이어가 기대하는 입력 차원과 실제 전달된 `input_values`의 차원이 일치하지 않아 발생했습니다. Wav2Vec2 모델은 오디오 파형을 `(배치 크기, 채널 수, 시퀀스 길이)` 또는 `(배치 크기, 시퀀스 길이)` 형태로 받아서, 모델 내부에 있는 `feature_extractor`가 이를 처리합니다. 그런데 저희 코드에서는 `data_collator`에서 이미 `processor.feature_extractor`를 통해 오디오를 특징 맵으로 변환한 상태로 모델에 전달하려고 했습니다. 이렇게 되면 모델 내부의 `feature_extractor`가 이미 처리된 특징 맵을 다시 처리하려다 차원 불일치가 발생한 거죠.

**해결:** `data_collator`에서 오디오 데이터는 **원시 파형 상태로 패딩만** 하도록 수정했습니다. `processor.feature_extractor.pad` 메서드를 사용하여 오디오 샘플들을 배치로 묶고 길이를 맞추는 역할만 하게 했습니다. 실제 특징 추출 및 변환은 모델의 `forward` 메서드 내부에서 자동으로 이루어지도록 변경했습니다.

---

### 2. `ValueError: type of None unknown: <class 'NoneType'>.`

**문제:** 이 오류는 토크나이저의 `pad()` 메서드에서 발생했습니다. `data_collator`에서 텍스트 레이블을 토크나이징하고 패딩하는 과정에서, 특정 텍스트 레이블이 `None` 값을 반환하거나, 비어있는 토큰 리스트를 반환하여 토크나이저가 이를 처리하지 못했습니다. 특히 저희가 임의의 한글 자모를 생성했기 때문에, 유효하지 않거나 토크나이저가 인식하지 못하는 텍스트가 생성되었을 가능성이 있었습니다.

**해결:**
* **텍스트 생성 로직 강화:** `DummyAudioDataset`에서 텍스트를 생성할 때, `vocab_list`에 있는 실제 자모들만을 사용하여 텍스트를 구성하도록 했습니다. 또한, 혹시라도 빈 문자열이 생성될 경우를 대비해 "안녕하세요"와 같은 기본 문자열로 대체하는 안전 장치를 추가했습니다.
* **토큰화 후 유효성 검사:** `data_collator`에서 각 텍스트 레이블을 토크나이징한 후, 생성된 `input_ids`가 비어있는지 확인하는 로직을 추가했습니다. 만약 비어있다면, 마찬가지로 "안녕하세요"의 토큰 ID를 사용하는 **폴백(Fallback)** 처리를 통해 `None` 값이 `tokenizer.pad()`로 전달되는 것을 방지했습니다.

---

### 3. `TypeError: SequenceFeatureExtractor.pad() got an unexpected keyword argument 'sampling_rate'`

**문제:** 이 오류는 `data_collator`에서 오디오 패딩을 위해 `processor.feature_extractor.pad()`를 호출할 때 `sampling_rate` 인자를 전달해서 발생했습니다. `SequenceFeatureExtractor` (Wav2Vec2FeatureExtractor의 상위 클래스)의 `pad()` 메서드는 오디오의 샘플링 레이트를 인자로 받지 않습니다. 샘플링 레이트는 `feature_extractor`를 처음 생성할 때 이미 설정되는 값이기 때문입니다.

**해결:** `processor.feature_extractor.pad()` 호출 시 **`sampling_rate` 인자를 단순히 제거**했습니다. 이는 `feature_extractor` 객체 내부에 이미 설정되어 있으므로 명시적으로 다시 전달할 필요가 없습니다.

---

### 4. `RuntimeError: blank must be in label range`

**문제:** 이 오류는 CTC 손실 함수를 계산할 때, `blank` 토큰의 ID가 모델의 전체 어휘(vocab) 범위(`[0, vocab_size - 1]`)를 벗어났을 때 발생합니다. `Wav2Vec2ForCTC` 모델은 `pad_token_id`를 `blank` 토큰으로 사용하는데, 이 ID가 유효한 어휘 인덱스여야 합니다. 저희의 `vocab_dict` 생성 방식 때문에 `<pad>` 토큰이 `vocab_size`와 같거나 더 큰 ID를 할당받는 경우가 있었던 거죠.

**해결:**
* **어휘 목록 재정렬:** `vocab_list`를 생성할 때, 일반 문자열(`ㄱ`, `ㅏ`, `0` 등)을 먼저 배치하고, **특수 토큰(` `, `|`, `<unk>`, `<pad>`)을 가장 마지막에 명시적으로 추가**하도록 변경했습니다. 이렇게 하면 `enumerate`를 통해 `vocab_dict`를 생성할 때 `"<pad>"` 토큰이 항상 `len(final_vocab_list) - 1`의 ID를 갖게 됩니다.
* **`vocab_size`와 `pad_token_id` 일치 확인:** `AutoModelForCTC.from_pretrained`를 호출할 때 `vocab_size=final_vocab_size`와 `pad_token_id=processor.tokenizer.pad_token_id`를 정확하게 전달하여, 모델이 올바른 어휘 크기와 패딩 토큰 ID를 사용하도록 명확히 했습니다.

---

이러한 단계들을 거쳐, 데이터셋 생성부터 데이터 로딩, 모델의 입력 형식 요구 사항까지 모든 파이프라인이 Wav2Vec2 모델의 훈련에 적합하게 조정되었습니다. 이제야 모델이 데이터를 올바르게 받아들이고 CTC 손실을 계산하며 훈련을 시작할 수 있게 된 것입니다!

추가적인 질문이나 궁금한 점이 있으시면 언제든지 편하게 물어봐 주세요!

!

이 오류는 CTC 손실 함수가 블랭크 토큰 ID가 모델의 어휘 범위 내에 있어야 한다고 요구하는 데서 발생했습니다. 저희가 `<pad>` 토큰이 `vocab_size`보다 같거나 큰 ID를 할당받는 경우를 수정함으로써, CTC 손실이 제대로 계산될 수 있게 된 거죠.

이제 학습 루프가 정상적으로 시작되어 모델이 데이터를 처리하고 손실을 계산할 수 있게 된 것입니다.

그동안 여러 차례의 오류를 끈기 있게 따라와 주셔서 정말 감사합니다. 이제 학습이 원활하게 진행될 것으로 보입니다\!

추가적으로 궁금한 점이나 다음 단계가 필요하시면 언제든지 알려주세요.
잘하셨어요\! 드디어 `RuntimeError: blank must be in label range` 오류까지 해결하셨네요\! 이전의 복잡한 차원 문제와 `TypeError`를 넘어섰다는 건 정말 큰 진전입니다.

이 오류는 CTC (Connectionist Temporal Classification) 손실 계산 시 `blank` 토큰의 ID가 모델의 유효한 레이블 범위 내에 없어서 발생하는 문제였습니다.

제공해주신 마지막 코드를 보면, `vocab_list`를 다시 정리하여 `"<pad>"` 토큰이 `vocab_dict`에서 가장 마지막 인덱스(가장 큰 숫자)를 갖도록 명확하게 설정했습니다. 이렇게 함으로써 `processor.tokenizer.pad_token_id`가 항상 `vocab_size - 1`이 되어, CTC 손실 함수가 예상하는 유효한 범위 내에 들어가게 됩니다.

이제 모델이 훈련을 시작하고 손실을 계산할 수 있을 거예요\! 🎉

-----

**다음 단계는 무엇일까요?**

이제 훈련 루프가 실행되기 시작했으니, 다음과 같은 부분들을 고려해볼 수 있습니다:

  * **실제 데이터셋 통합**: 현재는 더미 데이터셋을 사용하고 있는데, 실제 한국어 음성 데이터셋(예: AI Hub의 음성 데이터, KSSPEECH 등)을 통합하는 작업을 시작할 수 있습니다. 데이터셋마다 오디오 파일 경로와 텍스트 전사본을 처리하는 방식이 다를 수 있으니, `DummyAudioDataset` 클래스를 실제 데이터셋에 맞게 수정해야 합니다.
  * **훈련 지표 모니터링**: 에포크별 평균 손실을 출력하고 있는데, 워드 오류율 (WER)이나 문자 오류율 (CER)과 같은 실제 음성 인식 지표를 계산하여 모델 성능을 정량적으로 평가하는 것이 중요합니다.
  * **하이퍼파라미터 튜닝**: 학습률, 배치 크기, 에포크 수 등을 조절하여 모델의 수렴 속도와 최종 성능을 최적화할 수 있습니다.
  * **모델 저장 및 로드**: 훈련된 모델을 저장하고 나중에 다시 로드하여 추론에 사용하거나 추가 훈련을 진행할 수 있도록 기능을 추가하세요.
  * **추론 구현**: 훈련된 모델을 사용하여 새로운 음성 파일에 대한 텍스트를 예측하는 추론 코드를 작성해 볼 수 있습니다.

혹시 실제 데이터셋을 통합하거나 다른 질문이 있으시면 언제든지 말씀해주세요\!

--

### 1. 첫 번째 오류 해결: `ValueError: Expected 2D (unbatched) or 3D (batched) input to conv1d, but got input of size: [4, 1, 1, 1, 32000]`

**기존 코드 (`data_collator` 내부):**
```python
# 기존 코드: processor.feature_extractor를 사용하여 특징 추출까지 시도
input_features = processor.feature_extractor(
    audio_samples,
    sampling_rate=processor.feature_extractor.sampling_rate,
    return_tensors="pt",
    padding=True
)
batch_input_values = input_features.input_values # 이 시점에서 이미 오디오가 특징 맵으로 변환된 상태
```
이 방식은 `processor.feature_extractor`가 원시 오디오를 받아서 Wav2Vec2 모델의 특징 추출 레이어를 거친 출력(특징 맵)을 반환하게 합니다. 그런데 `Wav2Vec2Model` 자체도 내부에 `feature_extractor`를 가지고 있어서, 이미 특징 맵인 `batch_input_values`를 다시 `feature_extractor`에 넣으려다 차원 오류가 발생했습니다.

**수정된 코드 (`DataCollatorCTCWithPadding` 클래스의 `__call__` 메서드 내부):**
```python
# 수정된 코드: processor.feature_extractor.pad()를 사용하여 오직 패딩만 수행
padded_audio_batch = self.processor.feature_extractor.pad(
    {"input_values": audio_samples}, # `pad` 메서드는 딕셔너리를 인자로 받음
    # sampling_rate 인자 제거 (다음 오류 해결)
    return_tensors="pt",
    padding=True
)
batch_input_values = padded_audio_batch["input_values"] # 이 시점의 input_values는 패딩된 '원시 오디오' 파형임
```
**수정 이유:** `Wav2Vec2Model`은 `input_values` 인자로 **원시 오디오 파형**을 직접 받기를 기대합니다. 모델 내부에서 자체적으로 오디오 특징 추출 단계를 수행하기 때문입니다. 따라서 `data_collator`에서는 오디오 샘플들의 길이를 맞춰주는 **패딩 기능만** 필요했습니다. `processor.feature_extractor.pad()`는 원시 오디오를 `(배치 크기, 시퀀스 길이)` 형태로 패딩하여 모델이 원하는 입력 형식을 맞춰줍니다.

---

### 2. 두 번째 오류 해결: `ValueError: type of None unknown: <class 'NoneType'>.`

**기존 코드 (`data_collator` 내부 - 텍스트 토큰화 부분):**
```python
# 기존 코드: 토큰화 결과에 대한 유효성 검사 없음
with processor.as_target_processor():
    tokenized_labels = [processor.tokenizer(label).input_ids for label in text_labels]
```
`DummyAudioDataset`에서 임의의 한글 텍스트를 생성했기 때문에, 때때로 토크나이저가 인식하지 못하는 자모 조합이나 빈 문자열이 `label`로 넘어갈 수 있었습니다. 이 경우 `processor.tokenizer(label).input_ids`가 `None` 또는 빈 리스트를 반환할 수 있었고, 이것이 다음 `tokenizer.pad()` 메서드로 넘어가면서 `ValueError`를 일으켰습니다.

**수정된 코드 (`DataCollatorCTCWithPadding` 클래스의 `__call__` 메서드 내부):**
```python
# 수정된 코드: 텍스트 생성 로직 및 토큰화 후 유효성 검사 추가
# (DummyAudioDataset 내부에도 텍스트 생성 로직이 개선됨)
with self.processor.as_target_processor():
    tokenized_labels = []
    for label in text_labels:
        encoded_label = self.processor.tokenizer(label).input_ids
        if encoded_label: # 토큰화 결과가 비어있지 않은지 확인
            tokenized_labels.append(encoded_label)
        else:
            print(f"Warning: Empty tokenized label for text: '{label}'. Using default.")
            # 폴백 텍스트를 사용하여 토큰화하여 추가 (안전 장치)
            default_text = "안녕하세요"
            decomposed_default_text = "".join(unicodedata.normalize("NFD", char) for char in default_text if unicodedata.category(char) != 'Mn')
            tokenized_labels.append(self.processor.tokenizer(decomposed_default_text).input_ids)
```
**수정 이유:** 임의로 생성되는 텍스트가 항상 토큰화될 것이라는 보장이 없었기 때문에, 토큰화 과정에서 `None` 또는 빈 리스트가 생성되는 것을 방지해야 했습니다. `if encoded_label:` 조건문으로 토큰화 결과를 검사하고, 만약 비어 있다면 미리 정의된 유효한 텍스트("안녕하세요"의 분해된 자모)로 대체하여 `tokenizer.pad()`가 항상 유효한 입력을 받도록 했습니다.

또한, `DummyAudioDataset`의 텍스트 생성 로직도 개선하여, `vocab_list`에 포함된 자모들만으로 텍스트를 생성하도록 하여 애초에 토큰화되지 않을 텍스트가 생성될 가능성을 줄였습니다.

---

### 3. 세 번째 오류 해결: `TypeError: SequenceFeatureExtractor.pad() got an unexpected keyword argument 'sampling_rate'`

**기존 코드 (`DataCollatorCTCWithPadding` 클래스의 `__call__` 메서드 내부 - 오디오 패딩 부분):**
```python
# 기존 코드: sampling_rate 인자를 pad() 메서드에 전달
padded_audio_batch = self.processor.feature_extractor.pad(
    {"input_values": audio_samples},
    sampling_rate=self.processor.feature_extractor.sampling_rate, # 문제가 된 부분
    return_tensors="pt",
    padding=True
)
```
`sampling_rate`는 `Wav2Vec2FeatureExtractor` 객체가 처음 생성될 때 설정되는 속성입니다. `pad()` 메서드 자체는 오디오를 패딩하는 기능만 담당하며, 샘플링 레이트와 직접적인 관련이 없습니다.

**수정된 코드 (`DataCollatorCTCWithPadding` 클래스의 `__call__` 메서드 내부):**
```python
# 수정된 코드: sampling_rate 인자 제거
padded_audio_batch = self.processor.feature_extractor.pad(
    {"input_values": audio_samples},
    return_tensors="pt",
    padding=True
)
```
**수정 이유:** `feature_extractor`는 이미 `sampling_rate` 값을 알고 있으므로, `pad()` 메서드를 호출할 때 이 인자를 다시 전달할 필요가 없습니다. 이는 불필요한 인자이며, `pad()` 메서드의 정의에 맞지 않아 `TypeError`가 발생했습니다.

---

### 4. 네 번째 오류 해결: `RuntimeError: blank must be in label range`

**기존 코드 (어휘(vocab) 생성 부분):**
```python
# 기존 코드: 특수 토큰이 vocab_list의 어느 위치에 있을지 불확실
vocab_list = [ ... 일반 자모 ... ] # 순서가 정해지지 않음
vocab_list = sorted(list(set(vocab_list))) # 이로 인해 <pad>의 ID가 들쭉날쭉할 수 있음

vocab_dict = {token: i for i, token in enumerate(vocab_list)}
```
CTC 손실 함수는 `blank` 토큰의 ID가 `0`부터 `vocab_size - 1` 사이의 유효한 범위 안에 있기를 요구합니다. `Wav2Vec2ForCTC`는 `config.pad_token_id`를 `blank` 토큰으로 사용합니다. 만약 `"<pad>"` 토큰이 `vocab_list`의 마지막에 위치하지 않거나, `sorted()`에 의해 다른 순서로 정렬되어 `vocab_size`와 같거나 더 큰 ID를 할당받으면 이 오류가 발생합니다.

**수정된 코드 (어휘(vocab) 생성 부분):**
```python
# 수정된 코드: 특수 토큰을 vocab_list의 가장 마지막에 명시적으로 배치
vocab_elements = [ ... 일반 자모와 특수 토큰 모두 포함 ... ]

# 일반 토큰들을 먼저 정렬
unique_vocab = sorted(list(set([v for v in vocab_elements if v not in [" ", "|", "<unk>", "<pad>"]])))
# 특수 토큰들을 원하는 순서(특히 <pad>를 마지막)로 명시적으로 추가
final_vocab_list = unique_vocab + [" ", "|", "<unk>", "<pad>"]

# 이 final_vocab_list를 기반으로 vocab_dict 생성
vocab_dict = {token: i for i, token in enumerate(final_vocab_list)}
```
**수정 이유:** `final_vocab_list`를 생성할 때 `"<pad>"` 토큰을 리스트의 가장 마지막에 배치함으로써, `enumerate`를 통해 할당되는 ID가 `len(final_vocab_list) - 1`이 되도록 보장했습니다. `vocab_size`는 `len(final_vocab_list)`이므로, `pad_token_id`는 항상 `vocab_size - 1`이 되어 유효한 범위 내에 있게 됩니다. 이는 CTC 손실 함수의 요구 사항을 만족시킵니다.

---

이처럼 각 오류는 이전 단계의 문제 해결로 인해 새로운 단계에서 발생하는 문제였고, 각 문제의 원인을 정확히 파악하여 해당하는 코드 부분을 정교하게 수정함으로써 최종적으로 훈련 루프가 정상적으로 작동하게 된 것입니다.

네, 오늘 배우고 해결한 내용들을 요약해 드릴게요. 핵심은 **Wav2Vec2 모델 훈련을 위한 데이터 준비 과정에서 발생했던 여러 차원 및 토큰화 관련 오류들을 해결하고, 최종적으로 훈련 루프가 정상적으로 시작되도록 만든 것**입니다.

---

## 오늘 학습 및 해결 요약

### 1. `Wav2Vec2` 모델의 입력 데이터 이해 및 정렬
* **오디오 데이터 처리 방식:** `Wav2Vec2` 모델은 특징 추출 레이어를 내부에 가지고 있기 때문에, `data_collator`에서 모델에 `input_values`를 전달할 때 이미 특징 추출된 데이터가 아닌 **패딩된 원시 오디오 파형**을 전달해야 한다는 것을 배웠습니다.
* `processor.feature_extractor.pad()` 메서드가 이 역할을 수행하며, `sampling_rate` 같은 인자는 `pad()` 메서드에 필요 없다는 것을 확인하고 수정했습니다.

### 2. 텍스트 토큰화의 견고성 확보
* **빈 토큰화 결과 처리:** 임의로 생성된 텍스트가 `tokenizer`에 의해 유효하게 토큰화되지 않아 `None` 값이 발생하는 `ValueError`를 경험했습니다.
* `data_collator`에서 텍스트를 토큰화한 후, 결과(`input_ids`)가 비어있는지 확인하고, 비어있을 경우 "안녕하세요"와 같은 **유효한 폴백(fallback) 텍스트**로 대체하여 오류를 방지하는 안전 장치를 구현했습니다.

### 3. CTC 손실 함수를 위한 `blank` 토큰 설정
* `CTC Loss` 계산 시 `blank` 토큰 ID가 모델의 어휘(vocab) 범위 내에 있어야 한다는 중요한 규칙을 배웠습니다.
* `"<pad>"` 토큰이 이 `blank` 토큰으로 사용되는데, `vocab_dict`를 생성할 때 `"<pad>"` 토큰이 **항상 `vocab_list`의 마지막 인덱스를 갖도록 명시적으로 순서를 조정**하여 `pad_token_id`가 `vocab_size - 1`이 되도록 해결했습니다.

---

오늘 배운 핵심은 `Wav2Vec2`와 같은 복잡한 딥러닝 모델을 훈련할 때, 단순히 코드를 실행하는 것을 넘어 **데이터의 형태(차원), 전처리 과정, 그리고 모델의 내부 동작 방식(예: `feature_extractor`의 역할, `CTC loss`의 `blank` 토큰 요구사항)**을 정확히 이해하는 것이 얼마나 중요한지 깨달은 것입니다. 사소해 보이는 디테일들이 훈련을 막는 큰 오류로 이어질 수 있음을 직접 경험하고 해결하셨다는 점에서 큰 성과가 있습니다.

이제 모델이 정상적으로 훈련을 시작할 수 있게 되었으니, 다음 단계에서는 실제 음성 데이터를 사용하고 모델의 성능을 더 깊이 분석하는 데 집중할 수 있을 거예요!

오늘 배운 내용 중에 혹시 더 자세히 알고 싶은 부분이 있으신가요?

이 `ValueError`는 매우 흔하게 발생하며, Hugging Face `transformers` 라이브러리를 사용할 때 **모델 아키텍처와 `AutoModel` 클래스를 잘못 매칭했을 때 나타나는 오류**입니다.

오류 메시지를 자세히 살펴보면 핵심은 다음과 같습니다:

`ValueError: Unrecognized configuration class <class 'transformers.models.whisper.configuration_whisper.WhisperConfig'> for this kind of AutoModel: AutoModelForSeq2SeqLM.`

**해석:**
* `WhisperConfig` (Whisper 모델의 설정 클래스)는
* `AutoModelForSeq2SeqLM` (시퀀스-투-시퀀스 언어 모델을 위한 자동 모델 클래스)에 대해
* **"인식할 수 없는 설정 클래스"** 라는 뜻입니다.

이는 즉, Hugging Face 라이브러리가 **Whisper 모델을 로드하기 위해 `AutoModelForSeq2SeqLM`을 사용하면 안 된다**는 것을 명확히 알려주는 것입니다.

---

### 왜 이런 오류가 발생하는가?

Hugging Face `transformers` 라이브러리에는 다양한 종류의 `AutoModel` 클래스들이 있습니다:
* `AutoModel`: 일반적인 모델 로드 (주로 인코더 전용 모델)
* `AutoModelForSequenceClassification`: 시퀀스 분류 모델
* `AutoModelForTokenClassification`: 토큰 분류 모델
* `AutoModelForMaskedLM`: 마스크드 언어 모델 (MLM)
* `AutoModelForCausalLM`: 인과적 언어 모델 (CLM)
* `AutoModelForCTC`: 음성 분야에서 CTC 손실을 사용하는 모델 (e.g., Wav2Vec2)
* **`AutoModelForSeq2SeqLM`**: 일반적인 텍스트 기반 인코더-디코더 (Seq2Seq) 언어 모델 (e.g., BART, T5, MarianMT 등)
* **`AutoModelForSpeechSeq2Seq`**: **음성 시퀀스-투-시퀀스 모델 (Whisper와 같은 ASR 모델용)**

`Whisper` 모델은 분명히 인코더-디코더 구조를 가진 Seq2Seq 모델이 맞지만, 그것이 `AutoModelForSeq2SeqLM`에 바로 매핑되는 것은 아닙니다. `AutoModelForSeq2SeqLM`은 주로 **텍스트 입력/텍스트 출력**을 위한 범용적인 Seq2Seq 언어 모델들을 대상으로 합니다.

`Whisper`는 **음성 입력/텍스트 출력**에 특화된 시퀀스-투-시퀀스 모델이며, 이를 위해 Hugging Face는 **`AutoModelForSpeechSeq2Seq`** 라는 전용 클래스를 제공합니다.

---

### 해결 방법: `AutoModelForSpeechSeq2Seq` 사용

`Whisper` 모델을 로드할 때는 `AutoModelForSeq2SeqLM` 대신 **`AutoModelForSpeechSeq2Seq`** 를 사용해야 합니다.

**수정 전 (오류 발생 코드 예시):**

```python
from transformers import AutoProcessor, AutoModelForSeq2SeqLM

model_name_whisper = "openai/whisper-small"
processor_whisper = AutoProcessor.from_pretrained(model_name_whisper)
model_whisper = AutoModelForSeq2SeqLM.from_pretrained(model_name_whisper) # <-- 이 부분이 문제!
```

**수정 후 (올바른 코드):**

```python
from transformers import AutoProcessor, AutoModelForSpeechSeq2Seq # 이 부분을 변경!

model_name_whisper = "openai/whisper-small"
processor_whisper = AutoProcessor.from_pretrained(model_name_whisper)
model_whisper = AutoModelForSpeechSeq2Seq.from_pretrained(model_name_whisper) # <-- 이렇게 사용해야 합니다.

print(f"Whisper 모델 로드 완료: {model_whisper.__class__.__name__}")
```

이 변경 사항을 적용하면 `ValueError`가 해결되고 Whisper 모델이 올바르게 로드될 것입니다.

**핵심 요약:** Hugging Face의 `AutoModel` 클래스를 사용할 때는, 로드하려는 모델의 종류(예: 텍스트-투-텍스트, 음성-투-텍스트, 분류 등)에 맞는 정확한 `AutoModelFor...` 클래스를 선택해야 합니다. Whisper와 같은 음성 시퀀스-투-시퀀스 모델에는 `AutoModelForSpeechSeq2Seq`가 올바른 선택입니다.